# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here

# MATPLOTLIB for creating visualizations
# set retina to support width and height scaling (2X scaling)
# PANDAS for working with data structures
# NUMPY for efficient multi-dimensional array processing
# SEABORN for it's data visualation library based on matplotlib
# PYTORCH a framework for building and training neural networks 
# TORCHVISION is a package to load the data
# PIL for a number of functions to load images from files and to create new images



%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch import optim

import torchvision
from torchvision import datasets, models, transforms

from collections import OrderedDict

import PIL

import time





In [2]:
# SET DEBUG variable for Print Statements used for logging details
# DEBUG 1 is unused
# DEBUG 2 is unused
# DEBUG 3 is to print out the labels to flower mapping from the "cat_to_name" file
# DEBUG 4 is to print out the sizes of the datasets
# DEBUG 5 is unused
# DEBUG 6 is for the Feed-Forward section of code
# DEBUG 7 is for model input size selection

DEBUG = 7
print("\n*** Debug set to: {}  ****\n".format(DEBUG))



*** Debug set to: 7  ****



## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [3]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [4]:
# TODO: Define your transforms for the training, validation, and testing sets
# In general Transforms are common image transformations from torchvision
# Normalize the size of the images to 255x255
# Use Crop to take the center of the image 224x224
# ToTensor converts a Python Image Library (PIL) to a tensor or multi-dimensonal array
# Normalize a tensor image with a sequence of mean and standard deviations for each color channel (RGB)
# Add Random Rotations and Horizontal Flip to the TRAINING data

train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomRotation(30),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

valid_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

test_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
# Set Shuffle to True for the taining set
# Note: DataLoader is from TorchVision
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

   

In [5]:
# Print out the sizes of the datasets

if DEBUG == 6:
    print("Train_data_size", len(train_data))
    print("Valid_data_size", len(valid_data))
    print("Test_data_size", len(test_data))

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [6]:
# Use a JSON object to load in a mapping from category label to category name
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

#Print the mapping of label to name
if DEBUG == 3:
    print(cat_to_name)
print("\nLength:", len(cat_to_name))


Length: 102


# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [15]:
# TODO: Build and train your network
# Load a pre-trained network.  Start with vgg16, then add two other ones after vgg16 is working
# vgg16 is a 16 layer model

# Assign sizes to pretrained models dictionary:
# Key is the model name
# value[0] is the input size
# value [1] is the first hidden layer
# value [2] is the second hidden layer

pretrained_model_size = {"vgg16":[25088, 4096, 2048], 
                               "densenet121":[1024, 512, 256], 
                               "resnet18":[512, 256, 128]}

# Select the model

VGG = 0 
DENSENET = 0
RESNET = 1

if VGG:
    model = models.vgg16(pretrained=True)
    model.name = "vgg16"
    

elif DENSENET:
    model = models.densenet121(pretrained=True)
    model.name = "densenet121"

elif RESNET:
    model = models.resnet18(pretrained=True)
    model.name = "resnet18"

else:
    print("\nNo Model Selected. Please select a model")


    

input_size = pretrained_model_size[model.name][0]
hidden_layer_1 = pretrained_model_size[model.name][1]
hidden_layer_2 = pretrained_model_size[model.name][2]

if DEBUG == 7:
    print (model.name)
    print(input_size)
    print(hidden_layer_1)
    print(hidden_layer_2)

model  
                                       


resnet18
512
256
128


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [16]:
# Per the transfer learning lesson in UDACITY this codes will Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False


Some notes to follow when creating a NN

    The number of hidden neurons should be between the size of the input layer and the size of the output layer.
    The number of hidden neurons should be 2/3 the size of the input layer, plus the size of the output layer.
    The number of hidden neurons should be less than twice the size of the input layer.

In [17]:
# Define a new, untrained feed-forward network as a classifier

# Definitions:
# Fully Connect Layer 1 (fc1) has predifined inputs and output sizes per the model selected.  
# Using Rectified Linear Unit (relu) as the activation function per the training
# Dropout for layer 1 is used to avoid overfitting and helps to generalize the model
# fc2 input and output sizes are defined earlier as hidden_layer_1 and hidden_layer_2
# fc3 input size is predefined.  The output size of 102 is the final output size, aligned with number of categories in the image set.
# The output uses LogSoftmax per the training example


# Assign output size

output_size = 102

classifier = nn.Sequential(OrderedDict([
    ('fc1',nn.Linear(input_size, hidden_layer_1)),
    ('relu1', nn.ReLU()),
    ('dropout1', nn.Dropout(p=.5)),
    ('fc2', nn.Linear(hidden_layer_1, hidden_layer_2)), 
    ('relu2', nn.ReLU()),
    ('dropout2', nn.Dropout(p=.5)),
    ('fc3',nn.Linear(hidden_layer_2, output_size)),
    ('output', nn.LogSoftmax(dim=1))
]))    
  

model.classifier = classifier

#Set some print statements for debugging
if DEBUG == 6:
    print("Check FC1: ", classifier.fc1)
    print("Check FC2: ", classifier.fc2)
    print("Check FC2: ", classifier.fc3)
    print("Check FC1 Weight: \n ", classifier.fc1.weight)
    print("Check FC1 Bias: \n", classifier.fc1.bias)
    print("Check FC2 Weight: \n ", classifier.fc2.weight)
    print("Check FC2 Bias: \n", classifier.fc2.bias)
    print("Check FC3 Weight: \n ", classifier.fc2.weight)
    print("Check FC3 Bias: \n", classifier.fc2.bias)


In [18]:
# TODO: CUDA is 
# Automatically use CUDA if it's enabled
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [19]:
#change to device
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [20]:
# Next Step is to define the loss and optimizer


# Use Negative Log Likelihood Loss as criterion
criterion = nn.NLLLoss()


# Use Adam for the optimizer
# Per the UDACITY training: Adam is a variant of stochastic gradient descent which includes momentum and in general trains faster than your basic SGD.
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)


First, let's consider just one learning step before looping through all the data. The general process with PyTorch:

    Make a forward pass through the network to get the logits
    Use the logits to calculate the loss
    Perform a backward pass through the network with loss.backward() to calculate the gradients
    Take a step with the optimizer to update the weights


Now we'll put this algorithm into a loop so we can go through all the images. This is fairly straightforward. We'll loop through the mini-batches in our dataset, pass the data through the network to calculate the losses, get the gradients, then run the optimizer.

In [21]:
# Implement a function for the validation pass 
# From UDACITY Training: Inference & Validation


def validation(model, valid_loader, criterion):
    valid_loss = 0
    accuracy = 0
    
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)
        output = model.forward(images)
        valid_loss += criterion(output, labels).item()
        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    return valid_loss, accuracy     
    
    

In [22]:
# Train the classifier layers using backpropogation using the pre-trained network to get the frozen features
# Use Print statements throughout to show the start and stop points

# Define EPOCHs
# Print at various intervals during the batch runs to see the loss and accuracy
# Set a variable for steps through the optimizer
# Initialize running_loss
epochs = 5
print_every = 20
steps = 0
running_loss = 0


print("Training procuess initializing....\n")

for e in range(epochs):
    model.train()    
    for ii, (images, labels) in enumerate(train_loader):
        steps += 1
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        # Starting forward and backward passes on the training data
        
        outputs = model.forward(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()
            
            with torch.no_grad():
                valid_loss, accuracy = validation(model, valid_loader, criterion)
                
            print("Epoch:{}/{}".format(e+1, epochs),
             "Training Loss: {:.4f} | ".format(running_loss/print_every),
             "Validation Loss: {:.4f} | ".format(valid_loss/len(valid_loader)),
             "Validation Accuracy: {:.4f} | ".format(accuracy/len(valid_loader)))
            running_loss = 0
            model.train()
    
print("\nThe training process is now complete!!")
                




Training procuess initializing....



RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

Engineering Notes Regarding the Training
Batch Sizes
EPOCHS = 3
1. Set the initial batch size at 32,  EPOCHS at 3 and one hidden layer with size 4096.  Achieve 77.7% accuracy.  
2. Modify batch size to 64 to see if it has an effect on accuracy. 
3. At size 64, accuracy increased to 79.9%.
4. Let's increase to 96.  Achieved 80% at a batch size of 96.  
5. Let's reset it to 32 and see if there is another factor at work.  Down to 70% accuracy at batch of 32
6. Set now to 128. Achieve 83.5% accuracy.  So there appears to be a relationship between batch size and accuracy.

Set EPOCHS = 5 and two hidden layers with sizes 4096 and 2048. Set batch to 64.  Only 69% validation accuracy and 71% test accuracy.
Found error in code; move running_loss outside the for loop

Re-run EPOCHS = 5 and two hidden layers with sizes 4096 and 2048. Set batch to 64. Achieved 80% accuracy. 


In [17]:
## Testing your network

#It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [18]:
# TODO: Do validation on the test set
# Set a variable to store the accuracy
correct = 0

total = 0

with torch.no_grad():
    model.eval()
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _,predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy achieved by the network on test images is: %d%%'%(100*correct/total))

Accuracy achieved by the network on test images is: 71%


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes